In [0]:
# setting up config using storage account level SAS token access

account_name = "sttg1smartgear"     # My storage account on Azure
sas_token = "sv=2024-11-04&ss=b&srt=sco&sp=rwlcyx&se=2025-09-08T00:30:08Z&st=2025-09-07T16:15:08Z&spr=https&sig=zo1CiQrHS%2BQT0fUjhK4CSAmaddGMQvbcUUbow%2Bjvu80%3D"

spark.conf.set(f"fs.azure.account.auth.type.{account_name}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{account_name}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{account_name}.dfs.core.windows.net", sas_token)


In [0]:
# 1. Mounts storage
input_container = "rawdata"
curated_container="curated"
input_file_path = f"abfss://{input_container}@{account_name}.dfs.core.windows.net/raw/smartgear_sales.csv"
output_file_path = f"abfss://{curated_container}@{account_name}.dfs.core.windows.net/region_qty.parquet"

In [0]:
# 2. Reads the CSV with spark.read.csv
df = (spark.read
      .option("header", "true")     
      .option("inferSchema", "true")
      .csv(input_file_path))

df.show(10)

+-------+----------+------+-------+----------+--------+---------+
|OrderID| OrderDate|Region|StoreID|   Product|Quantity|UnitPrice|
+-------+----------+------+-------+----------+--------+---------+
|   1001|2025-03-14|  East|    115|Headphones|       3|    81.65|
|   1002|2025-02-19|  West|    110|Smartwatch|       3|    242.5|
|   1003|2025-03-17|  West|    113|   Printer|       2|   152.59|
|   1004|2025-01-05|  West|    118|    Camera|       4|    463.4|
|   1005|2025-01-07|  West|    110|    Tablet|       3|   370.01|
|   1006|2025-03-22|  East|    113|Smartphone|       4|   639.47|
|   1007|2025-02-19| North|    118|     Drone|       4|   747.21|
|   1008|2025-02-21|  West|    112|   Printer|       2|   151.06|
|   1009|2025-03-30| North|    117|Smartphone|       1|   614.89|
|   1010|2025-01-30|  West|    108|    Tablet|       3|   415.43|
+-------+----------+------+-------+----------+--------+---------+
only showing top 10 rows


In [0]:
# 3.  Aggregating total Quantity by Region
import pyspark.sql.functions as func
region_qty = (df.groupBy("Region")
                .agg(func.sum("Quantity").alias("total_quantity"))
                .orderBy(func.desc("total_quantity")))

region_qty.show()

+------+--------------+
|Region|total_quantity|
+------+--------------+
| North|           848|
| South|           740|
|  East|           719|
|  West|           694|
+------+--------------+



In [0]:
# writing result output from dataframe to parquet
region_qty.coalesce(1).write.mode("overwrite").parquet(output_file_path)
print("output result saved as region_qty.parquet in a container named - curated")

output result saved as region_qty.parquet in a container named - curated
